<a href="https://colab.research.google.com/github/MohsenMaaleki/CIFAR_10_Objects_Classification/blob/main/CIFAR_10_Objects_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms

In [16]:
# Load the pre-trained ResNet18 model
model = torchvision.models.resnet18(pretrained=True)

# Freeze all layers in the feature extractor
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer with a fully connected layer for 10 classes
model.fc = torch.nn.Linear(512, 10)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [25]:
# Load the CIFAR-10 dataset and perform data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
# Train the model on the CIFAR-10 dataset
for epoch in range(10):  # Loop through 10 epochs

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):  # Loop through the data in the training set
        inputs, labels = data  # Get the inputs and labels from the data

        optimizer.zero_grad()  # Zero the gradient buffers

        outputs = model(inputs)  # Get the model outputs
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Perform backpropagation
        optimizer.step()  # Update the parameters of the model

        running_loss += loss.item()  # Add up the loss for each iteration

    avg_loss = running_loss / len(trainloader)  # Calculate the average loss for each epoch
    print('[Epoch %d] loss: %.3f' % (epoch + 1, avg_loss))  # Print the average loss for each epoch


[Epoch 1] loss: 1.646
[Epoch 2] loss: 1.645
[Epoch 3] loss: 1.646
[Epoch 4] loss: 1.645
[Epoch 5] loss: 1.643
[Epoch 6] loss: 1.639
[Epoch 7] loss: 1.638
[Epoch 8] loss: 1.643
[Epoch 9] loss: 1.643
[Epoch 10] loss: 1.647


In [27]:
# Evaluate the model on the CIFAR-10 test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the model on the test set: %d %%' % (100 * correct / total))


Accuracy of the model on the test set: 41 %
